# purpose

- Emily setting up the lab's dj
- later tailor to show how to connect to dj for new users

# imports

In [27]:
import datajoint as dj
import glob, os, sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import getpass

In [31]:
!pip install element_session

  Using cached element_session-0.1.2-py3-none-any.whl (6.8 kB)


In [34]:
sys.path.append(os.path.dirname(os.getcwd()))
from src import *
from src.workflow.workflow import lab, subject, session
from src.workflow.ingest import ingest_lab, ingest_subjects, ingest_sessions

# procedure

In [7]:
# connect to dj

dj.config['database.host'] = 'vm7146.int.janelia.org'
dj.config['database.user'] = 'djAdmin'  # enter your username
dj.config['database.password'] = getpass.getpass() # enter the password securily
dj.conn(use_tls=True)


········


DataJoint connection (connected) djAdmin@vm7146.int.janelia.org:3306

In [8]:
#using tutorial from tutorials.datajoint.io/beginner/building-first-pipeline/python/first-table.html, which only opens in microsoft edge browser for me 2022/08/10
schema = dj.schema('all_dennislab',locals())
# adding the locals() allows us to see all other tables created and call them simply by their name

In [ ]:
### prefixes are useful - like dennis_

if 'custom' not in dj.config:
    dj.config['custom'] = dict()
dj.config['custom']['database.prefix'].append('all_')
dj.config.save_local() #should make dj_local_conf.json

# old procedure to borrow from

In [14]:
#ok let's use the schema object to create a new table, mouse
# we "decorate" the class Mouse with the schema object, meaning that Mouse table is specified inside the schema object that points to tutorial.
# we use dj.Manual because we're going to manually enter this info
# the # mouse line gives us space for a reader-friendly comment about what the table is/contains
# the rest is formatted as attribute: datatype # comment
# the --- separates primary keys (UNIQUE identifiers) and secondary keys (can be repeated)
@schema
class Mouse(dj.Manual):
    definition = """
    # mouse
    mouse_id: int    #unique mouse id, an integer
    ---
    dob: date   #mouse date of birth, a date 
    sex_gonadal: enum('M','F','I','U') #gonadal sex, male, female, intersex, unclassified or unknown
    """

In [16]:
mouse = Mouse()
mouse.insert1( (0,'2022-08-10','F'))

In [17]:
mouse

"mouse_id unique mouse id, an integer","dob mouse date of birth, a date","sex_gonadal gonadal sex, male, female, intersex, unclassified or unknown"
0,2022-08-10,F


In [20]:
data_dict = {
    'mouse_id': 1,
    'dob': '2022-08-10',
    'sex_gonadal':'U'
}

In [21]:
mouse.insert1(data_dict)

In [22]:
mouse

"mouse_id unique mouse id, an integer","dob mouse date of birth, a date","sex_gonadal gonadal sex, male, female, intersex, unclassified or unknown"
0,2022-08-10,F
1,2022-08-10,U


In [23]:
data_dict_2 =  {
    'mouse_id': 2,
    'dob': '2022-08-10',
    'sex_gonadal':'M'
}
data_dict_3 =  {
    'mouse_id': 3,
    'dob': '2022-08-10',
    'sex_gonadal':'I'
}
mouse.insert([data_dict_2,data_dict_3])
mouse

"mouse_id unique mouse id, an integer","dob mouse date of birth, a date","sex_gonadal gonadal sex, male, female, intersex, unclassified or unknown"
0,2022-08-10,F
1,2022-08-10,U
2,2022-08-10,M
3,2022-08-10,I


# Borrowed from Brody and IBL

In [27]:
# add graphviz to path, taken from https://github.com/Brody-Lab/bl_pipeline_python/blob/main/notebooks/00-Datajoint-configuration.ipynb
if os.name == 'nt':
    if 'C:\\Program Files\\Graphviz\\bin' not in os.environ["PATH"]:
        os.environ["PATH"] += ';C:\\Program Files\\Graphviz\\bin'

In [32]:
# later we can make this into a standalone file in a repo, and import it


# return to procedure

In [55]:
# defining a table module
# this is making a .py file that is read as a module in python so we can import our tables


for_file = ["import datajoint as dj",
            "schema = dj.schema('tutorial',locals())",    
            "@schema","class Mouse(dj.Manual):",
            "    definition = \"\"\"",
            "    # mouse",
            "    mouse_id: int    #unique mouse id, an integer",
            "    ---",
            "    dob: date   #mouse date of birth, a date", 
            "    sex_gonadal: enum('M','F','I','U') #gonadal sex, male, female, intersex, unclassified or unknown\"\"\""]


def make_table_def_as_file(file_loc,table_data):
    """ takes a location and a list of lines to write to a table .py file"""
    if not os.path.exists(os.path.dirname(file_loc)):
        print("WARNING invalid path, {} does not exist".format(os.path.dirname(file_loc)))
    if "py" not in file_loc:
        print("adding .py to file_loc")
        file_loc = "{}.py".format(file_loc)
    print("saving table at {}".format(file_loc))
    with open(file_loc,"w") as f:
        for line in table_data:
            f.write("{}\n".format(line))
        f.close()
        
file_loc = 'tutorial_tables.py'
make_table_def_as_file(file_loc,for_file)

WARNING invalid path,  does not exist
saving table at tutorial_tables.py


In [56]:
from tutorial_tables import *

In [57]:
# fetch some data
data = mouse.fetch()
data

array([(0, datetime.date(2022, 8, 10), 'F'),
       (1, datetime.date(2022, 8, 10), 'U'),
       (2, datetime.date(2022, 8, 10), 'M'),
       (3, datetime.date(2022, 8, 10), 'I')],
      dtype=[('mouse_id', '<i8'), ('dob', 'O'), ('sex_gonadal', 'O')])

In [58]:
# fetch some data as a dict
data = mouse.fetch(as_dict=True)
data

[{'mouse_id': 0, 'dob': datetime.date(2022, 8, 10), 'sex_gonadal': 'F'},
 {'mouse_id': 1, 'dob': datetime.date(2022, 8, 10), 'sex_gonadal': 'U'},
 {'mouse_id': 2, 'dob': datetime.date(2022, 8, 10), 'sex_gonadal': 'M'},
 {'mouse_id': 3, 'dob': datetime.date(2022, 8, 10), 'sex_gonadal': 'I'}]

In [62]:
# another way to fetch data
sex, ids = mouse.fetch('sex_gonadal','mouse_id')
print(sex, ids)

mouse & 'sex_gonadal = "M"'

['F' 'U' 'M' 'I'] [0 1 2 3]


"mouse_id unique mouse id, an integer","dob mouse date of birth, a date","sex_gonadal gonadal sex, male, female, intersex, unclassified or unknown"
2,2022-08-10,M


In [47]:
# add users

## use elements
https://www.youtube.com/watch?v=NRqpKNoHEY0 at 4:57
want to have a workflow_session folder, with files including

```
__init__.py
export.py
ingest.py
paths.py
workflow.py
version.py
```

and in workflow.py, it imports the multiple schemas using 

```
import datajoint as dj
from element_lab import lab

if 'custom' not in dj.config:
    dj.config['custom'] = {}
    
db_prefix = dj.config['custom']..get('database.prefix','')

lab.activate(db_prefix + 'lab')
```

In [68]:
# import elements
from element_lab import lab

In [73]:
lab.activate('schema')
lab.Lab()

lab Abbreviated lab name,lab_name full lab name,institution,address,time_zone 'UTC±X' format for NWB export


In [ ]:
# our data structure